In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

/home/com03/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Quiz 1: Cross-boundary anchors
    
![alt text](bbox_clip.jpg "Title")

### complete clip operations so that 'clib_boxes_graph' function operates as follows
<br>
#clipped = clip_boxes_graph(boxes, window) <br>
#sess.run(clipped) <br>
array([[  0,   0, 300, 300], <br>
      [250, 250, 450, 450], <br>
      [500, 600, 599, 700]], dtype=int32)

In [43]:
def clip_boxes_graph(boxes, window):   ### window를 기준으로 box의 튀어 나온 부분을 보정
    """ 
    boxes: [N, 4] where N boxes are specified in the form (y1, x1, y2, x2) 
    window: [4] in the form y1, x1, y2, x2 
    """

    # Split
    wy1, wx1, wy2, wx2 = tf.split(window, 4)
    y1, x1, y2, x2 = tf.split(boxes, 4, axis=1)
    
    print(boxes.shape)
    sess.run(wy1)
    print(wy1.shape)    
    print(wx1.shape)
    print(y2.shape)    
    print(x2.shape)
    
    
    # implement clip operation using min / max operation # 
    
    # tf.clip_by_value 로 구현 필요
    
    y1 = tf.cond(y1 < wy1, lambda: tf.maximum(y1,wy1), lambda: y1)
    x1 = tf.cond(x1 < wx1, lambda: tf.maximum(x1,wx1), lambda: x1)
    
    y1 = tf.cond(y2 > wy2, lambda: tf.minimum(y2,wy2), lambda: y2)
    x1 = tf.cond(x2 > wx2, lambda: tf.minimum(x2,wx2), lambda: x2)

    
    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    clipped.set_shape((clipped.shape[0], 4))
    return clipped

In [41]:
window = tf.constant([0,0,599,799])
boxes = tf.constant([[-10, -10, 300, 300],
                     [250, 250, 450, 450],
                     [500, 600, 800, 700]])

In [42]:
boxes = tf.clip_by_value(boxes,0,700)
sess.run(boxes)

array([[  0,   0, 300, 300],
       [250, 250, 450, 450],
       [500, 600, 700, 700]], dtype=int32)

In [34]:
clipped = clip_boxes_graph(boxes, window)

(3, 4)
(1,)
(1,)
(3, 1)
(3, 1)


ValueError: Shape must be rank 0 but is rank 2 for 'cond_7/Switch' (op: 'Switch') with input shapes: [3,1], [3,1].

In [5]:
sess.run(clipped)

array([[  0,   0, 300, 300],
       [250, 250, 450, 450],
       [500, 600, 599, 700]], dtype=int32)

---

# Quiz 2: NMS(non-maximum suppression)

![alt text](nms.jpg "Title")

### insert values of "scores" tensor so that nms function outputs <br>

#sess.run(proposals) <br>
array([[  1.,   1., 298., 299.]], dtype=float32)

In [6]:
# hint: https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression?hl=en
def nms(boxes, scores):
    indices = tf.image.non_max_suppression(
        boxes, scores, 1, name="rpn_nms")
    proposals = tf.gather(boxes, indices)
    
    # Pad if needed
    padding = tf.maximum(1-tf.shape(proposals)[0], 0)
    proposals = tf.pad(proposals, [(0, padding), (0, 0)])
    return proposals

In [7]:
boxes = tf.constant([[0., 0., 300., 300.],
                     [0., 0., 301., 299.],
                     [1., 1., 298., 299.]])
scores = tf.constant([0., 0., 1.])  ############################ Answer

In [8]:
proposals = nms(boxes, scores)

In [9]:
sess.run(proposals)

array([[  1.,   1., 298., 299.]], dtype=float32)

---

# Quiz 3: Smooth L1 Loss

![alt text](smooth_l1.JPG "Title")

### implement Smooth L1 Loss

In [10]:
def smooth_l1_loss(prediction, target):
    prediction = tf.to_float(prediction)
    target = tf.to_float(target)
    
    # implement smooth_l1_loss calculating code below
    #abs_diff = ...
    #loss = ...
    return loss

In [11]:
prediction1 = tf.constant(3)
target1 = tf.constant(5)
loss1 = smooth_l1_loss(prediction1, target1)

prediction2 = tf.constant(3)
target2 = tf.constant(3.1)
loss2 = smooth_l1_loss(prediction2, target2)

In [12]:
sess.run([loss1, loss2])

[1.5, 0.0049999906]

----

# Quiz 4: Binary Cross-Entropy Loss

![alt text](bce.jpg "Title")

In [1]:
def binary_cross_entropy_loss(prediction, target, eps=1e-9):    ### target = P*, prediction = P
    '''
    prediction : (N, 2)-shaped, float tensor
    target : (N)-shaped, float tensor
    eps : very small number for numerical stability
    '''
    # implement BCE here
    #loss = ...
    return loss

In [14]:
prediction1 = tf.constant(1.)
target1 = tf.constant(1.)
loss1 = binary_cross_entropy_loss(prediction1, target1)

prediction2 = tf.constant(0.)
target2 = tf.constant(1.)
loss2 = binary_cross_entropy_loss(prediction2, target2)

prediction3 = tf.constant(0.8)
target3 = tf.constant(1.0)
loss3 = binary_cross_entropy_loss(prediction3, target3)

In [15]:
sess.run([loss1, loss2, loss3])

[0.0, 20.723267, 0.22314353]

---

# Quiz 5: IoU (Intersection-over-Union)

![alt text](iou.jpg "Title")

In [16]:
def iou(box1, box2):    ################## tf.cond 사용
    '''
    box1 : (4)-shaped float tensor. (b1_y1, b1_x1, b1_y2, b1_x2)
    box2 : (4)-shaped float tensor. (b2_y1, b2_x1, b2_y2, b2_x2)
    '''
    
    b1_y1, b1_x1, b1_y2, b1_x2 = tf.split(box1, 4)
    b2_y1, b2_x1, b2_y2, b2_x2 = tf.split(box2, 4)
    
    # implement iou here
    #box1_area = ...
    #box2_area = ...
    
    #d_y1 = ...
    #d_x1 = ...
    #d_y2 = ...
    #d_x2 = ...
    
    #intersection = ...
    #union = ...
    
    score = intersection/union
    return score

In [17]:
box1 = tf.constant([0.,0.,300.,300.])
box2 = tf.constant([100.,100.,400.,400.])

In [18]:
overlap_score = iou(box1, box2)

In [19]:
sess.run(overlap_score)

array([0.2857143], dtype=float32)

# Quiz 6 : Bbox refinement via regression

![alt text](bbox_regression.jpg "Title")

In [20]:
def refine_bbox(anchor_box, regression):
    '''
    anchor_box : (4)-shaped anchor bbox coordinates (y1, x1, y2, x2)
    regression : (4)-shaped regressed parameter for bbox refinement (dy, dx, sy, sx)
    '''
    y1, x1, y2, x2 = tf.split(anchor_box, 4)
    dy, dx, sy, sx = tf.split(regression, 4)
    
    # implement refine code here
    #h = ...
    #w = ...
    
    #new_y1 = ...
    #new_x1 = ...
    #new_y2 = ...
    #new_x2 = ...
    
    refined_box = tf.concat([new_y1, new_x1, new_y2, new_x2], axis=0)
    return refined_box    

In [25]:
anchor_box = tf.constant([0.,0.,300.,300.])
regression = tf.constant([5, 5, 0.5, 0.5])

In [26]:
refined_box = refine_bbox(anchor_box, regression)

In [27]:
sess.run(refined_box)

array([  5.,   5., 155., 155.], dtype=float32)